In [1]:
import os
import json
import shutil
import lzma

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

C:\Users\rdm42\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [84]:
download_dir = os.path.join('..','0_data','downloaded_posts')
metadata_paths = [os.path.join(download_dir,f) for f in os.listdir(download_dir) if f.endswith('.json.xz')]

additional_data = []

for i,path in enumerate(metadata_paths):

    with lzma.open(path, 'rt') as file:
        data_dict = json.load(file)
    
    short = data_dict['node']['shortcode']
    user  = data_dict['node']['owner']['username']
    likes = data_dict['node']['edge_media_preview_like']['count']
    caption = data_dict['node']['edge_media_to_caption']['edges'][0]['node']['text']
    long = os.path.split(path)[-1].split('.')[0]

    additional_data.append((short,user,likes,caption,long))

additional_data = pd.DataFrame(additional_data, columns=['Short','user','likes','caption','Long']) \
                    .drop(columns=['Long'])
additional_data['URL'] = additional_data.Short.apply(lambda code: f"https://www.instagram.com/p/{code}")
additional_data.head(3)

,Short,user,likes,caption,URL
0,fNO52Nxluy,amenstyle,61,Valentina Vignali looks explosive in a preciou...,https://www.instagram.com/p/fNO52Nxluy
1,lzQl_gRlnD,amenstyle,374,Regram @nimabenati #amenstyle #amen #fashion #...,https://www.instagram.com/p/lzQl_gRlnD
2,mrzAb-Rlkt,amenstyle,175,RG @nimabenati #amencouture amenstyle.com,https://www.instagram.com/p/mrzAb-Rlkt


In [80]:
with pd.ExcelFile('pictures_captions_autocaptions.xlsx') as xl:
    autocaptions = xl.parse()[['Short','Auto_caption']]
    
# autocaptions['Long'] = autocaptions.Long.str.extract("(.*_UTC)")[0]

In [87]:
output_table = autocaptions.merge(additional_data, on='Short', how='left', validate='m:1') \
               [['Short','URL','user','likes','caption','Auto_caption']]

In [90]:
output_table.to_clipboard(index=False)

In [97]:
output_table[output_table.duplicated(keep=False)]

,Short,URL,user,likes,caption,Auto_caption
168,CiaXyUYsqcJ,https://www.instagram.com/p/CiaXyUYsqcJ,jacquemus,188018.0,Bella in Jacquemus 🏹🫠,a picture of a woman with long hair wearing a ...
170,CiaXyUYsqcJ,https://www.instagram.com/p/CiaXyUYsqcJ,jacquemus,188018.0,Bella in Jacquemus 🏹🫠,a picture of a woman with long hair wearing a ...
